## Projet d'optimisation ##

In [65]:
import numpy as np
from scipy import optimize
from casadi import *
import time

# Question 1

Le but du problème est de trouver l'allocation globale $x \in \mathbb{R}^m$ qui minimise le risque associé au rendement du portefeuille, i.e. la variance de cette allocation par rapport à la variation du prix des actifs $p \in \mathbb{R}^m$.

La variance est l'équivalent d'un "écart à la moyenne" ; elle représente donc bien un indicateur du risque associé à un investissement. Ce risque prend en compte le lien entre la variation des prix et les allocations, d'où $x^T \Sigma x$, où $\Sigma$ est la matrice de covariance associé à $p$.

La relation $\bold{1}^T x = 1$ signifie simplement que l'on normalise l'allocation par souci de cohérence et de clarté ; pour une allocation donnée, le rendement représente le gain moyen associé à l'allocation, d'où $\bar p^T x = r$.

# Question 2

On souhaite minimiser le risque, donc on a
$$
\begin{array}{rcl}
f : z & \longmapsto & f(z) = x^T \Sigma x
\end{array}
$$

La variable $z$ dépend donc de $x$, mais aussi de $\Sigma = \left[ \mathrm{Cov}(p_i, p_j) \right]_{i,j \in \llbracket 1, m \rrbracket}$ et donc de $p$. Le nombre de variables de décisions est donc $n = 2$, où chaque variable est un vecteur de $\mathbb{R}^m$.

L'énoncé contraint le problème par le biais de la condition de normalisation $\bold{1}^T x = 1$ et celle de rendement $r = \bar p^T x$ : on peut donc définir $$c_{eq,norm}(z) = \bold{1}^T x - 1$$ et $$c_{eq,rend}(z) = \bar p^T x - r$$ ce qui ajoute les contraintes égalité $c_{eq} : \begin{cases} c_{eq,norm}(z) = 0 \\ c_{eq,rend}(z) = 0 \end{cases}$ au problème de minimisation de $f$.

# Question 3

On ajoute une contrainte inégalité pour limiter les positions _short_ à un certain montant par allocation, afin de garantir un équilibre de marché, ne pas entraîner de trop fortes fluctuation des actifs et donc une variance trop élevée.

On a donc dans ce problème une contrainte égalité (représentée par la fonction $c_{eq}$) et une contrainte inégalité.

Or la contrainte inégalité comprend déjà une fonction $max$ ; on cherche donc à minimiser une fonction $f$ sous contrainte d'une limitation d'un maximum, ce qui risque de compliquer les calculs...

# Question 4

$\max(-x,\bold{0})$ vérifie
$\forall x \in \mathbb{R}^m, \max(-x,\bold{0}) ≥ -x$ et $\max(-x,\bold{0}) ≥ 0$.

En introduisant $s \in \mathbb{R}^m$ tel que $s ≥ -x$, $s ≥ 0$ et $\bold{1}^T s ≤ s_M$, on a 
$$\bold{1}^T \max(-x,\bold{0}) ≤ \bold{1}^T s ≤ s_M$$
En prenant $s$ le plus proche possible de $\max(-x,\bold{0})$, on peut reformuler de manière équivalente le problème de la question 2 :
$$\min f(z)$$
sous les contraintes $c_{eq}(z) = 0$, $c_{in}(s) ≤ 0$ 

avec $\begin{cases} s ≥ -x \\ s ≥ 0 \end{cases}$ et $c_{in}(s) = \bold{1}^T s - s_M$

La variable $s$ est indirectement liée à $x$, et donc à $z$, puisqu'elle est choisie en fonction de $-x$ (on veut avoir $s = \max(-x, \bold{0})$ en se débarrassant du max).

# Question 5

Le problème $(4)$ est un problème d'optimisation sous contrainte avec une contrainte égalité affine.

Une première méthode de résolution est d'utiliser les multiplicateurs de Lagrange, en calculant le lagrangien
$$\mathcal{L}(z, \lambda) = f(z) + \lambda^T c_{eq}(z)$$
puis en  trouvant ses points stationnaires.

On peut aussi penser aux conditions $(KKT)$ : en effet, une contrainte affine étant convexe, on a une équivalence entre respecter les conditions $(KKT)$ et être solution du problème d'optimisation sous contrainte affine.

# Question 6

Pour résoudre numériquement le problème, on va utiliser les bibliothèques Scipy (avec son module _optimize_ et son solveur SLSQP) et Casadi (avec _ipopt_).

In [66]:
# Définition des variables
p1 = 0.05
p2 = 0.15
p3 = 0.3
p = np.array([p1, p2, p3])
rho = 0.1
r = 0.1

sig1 = 0.1
sig2 = 0.3
sig3 = 0.8
sigma = np.array([[sig1**2, rho*sig1*sig2, 0], [rho*sig1*sig2, sig2**2, 0], [0, 0, sig3**2]])

unit = np.ones(3)
x0 = np.array([0.3, 0.2, 0.5]) #pour remplir la condition de normalisation

In [67]:
# Définition de la fonction f et des contraintes

def var(x):
    return x.T @ sigma @ x

contraintes = ({'type': 'eq', 'fun': lambda x:  unit.T@x -1},
        {'type': 'eq', 'fun': lambda x: p.T@x - r})

In [68]:
# BFGS avec Scipy
print('méthode SLSQP avec Scipy (r = 0.1)')
start_time = time.time()
res = optimize.minimize(var, x0, method='SLSQP', jac=None, constraints=contraintes, options={'disp': True})
print("Allocation :", res.x)
print("Durée : ", time.time()-start_time)

méthode SLSQP avec Scipy (r = 0.1)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.01706777546777549
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Allocation : [0.63284823 0.27858629 0.08856549]
Durée :  0.007384777069091797


In [69]:
# Résolution avec Casadi
opti_cons = casadi.Opti();
x = opti_cons.variable(3)

opti_cons.minimize(var(x))
opti_cons.subject_to(dot(unit.T, x) - 1 == 0)
opti_cons.subject_to(dot(p.T, x) - r == 0)

opti_cons.set_initial(x,x0)
opti_cons.solver('ipopt')

print("Résolution avec Casadi...")
start_time = time.time()
sol = opti_cons.solve()
print("Durée : ", time.time()-start_time)
print(sol.value(x))

Résolution avec Casadi...
This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6486000e-01 9.50e-02 1.46e-01  -1.0 0.00e+0

On remarque que le temps de calcul avec Scipy et Casadi sont sensiblement les mêmes (de l'ordre de $10^{-3}$ s)

De plus, les 2 méthodes aboutissent au même résultat, à savoir une variance de l'ordre de $10^{-2}$.

# Question 7

$(a)$

$\rho$ est le coefficient de corrélation associé aux vecteurs $p_1$ et $p_2$. Il peut prendre des valeurs dans l'intervalle $[-1, 1]$.

$(b)$

In [70]:
# définition des variables


$(c)$

On ne résout qu'avec la méthode SLSQP de Scipy à présent pour plus de clarté, on n'utilise plus Casadi mais la réponse reste la même.

In [71]:
# rho = 0.5
rho1 = 0.5
sigma1 = np.array([[sig1**2, rho1*sig1*sig2, 0], [rho1*sig1*sig2, sig2**2, 0], [0, 0, sig3**2]])

def var1(x):
    return x.T @ sigma1 @ x

print('méthode SLSQP avec Scipy (rho = 0.5)')
start_time = time.time()
res = optimize.minimize(var1, x0, method='SLSQP', jac=None, constraints=contraintes, options={'disp': True})
print("Allocation :", res.x)
print("Durée : ", time.time()-start_time)

méthode SLSQP avec Scipy (rho = 0.5)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.021123595533492193
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Allocation : [0.65168539 0.24719102 0.10112359]
Durée :  0.0017781257629394531


In [72]:
# rho = -0.5
rho2 = -0.5
sigma2 = np.array([[sig1**2, rho2*sig1*sig2, 0], [rho2*sig1*sig2, sig2**2, 0], [0, 0, sig3**2]])

def var2(x):
    return x.T @ sigma2 @ x

print('méthode SLSQP avec Scipy (rho = -0.5)')
start_time = time.time()
res = optimize.minimize(var2, x0, method='SLSQP', jac=None, constraints=contraintes, options={'disp': True})
print("Allocation :", res.x)
print("Durée : ", time.time()-start_time)

méthode SLSQP avec Scipy (rho = -0.5)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.01039252337681496
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Allocation : [0.60934579 0.31775702 0.07289719]
Durée :  0.0018308162689208984


SPAP analyse de la question précédente :/

# Question 8

$(a)$

In [73]:
# Pour r = 0.2
r1 = 0.2

contraintes1 = ({'type': 'eq', 'fun': lambda x:  unit.T@x -1},
        {'type': 'eq', 'fun': lambda x: p.T@x - r1})

print('  Méthode SLSQP avec Scipy (r = 0.2)')
start_time = time.time()
res = optimize.minimize(var, x0, method='SLSQP', jac=None, constraints=contraintes1, options={'disp': True})
print("Allocation :", res.x)
print("Durée : ", time.time()-start_time)

  Méthode SLSQP avec Scipy (r = 0.2)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.10745197508500445
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Allocation : [-0.08274429  0.80457381  0.27817048]
Durée :  0.0018529891967773438


L'optimum passe ici par une allocation négative dans le _set_ total !

In [83]:
# pour r = 0.15
r2 = 0.15

contraintes2 = ({'type': 'eq', 'fun': lambda x:  unit.T@x -1},
        {'type': 'eq', 'fun': lambda x: p.T@x - r2})

print('  Méthode SLSQP avec Scipy (r = 0.15)')
start_time = time.time()
res = optimize.minimize(var, x0, method='SLSQP', jac=None, constraints=contraintes2, options={'disp': True})
print("Allocation :", res.x)
print("Durée : ", time.time()-start_time)

  Méthode SLSQP avec Scipy (r = 0.15)
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.04956735970798472
            Iterations: 4
            Function evaluations: 16
            Gradient evaluations: 4
Allocation : [0.27505198 0.54158003 0.18336799]
Durée :  0.005079030990600586


In [75]:
'''
# équivalent avec Casadi pour r = 0.2

opti_cons1 = casadi.Opti();
x1 = opti_cons1.variable(3)

opti_cons1.minimize(var(x1))
opti_cons1.subject_to(dot(unit.T, x1) - 1 == 0)
opti_cons1.subject_to(dot(p.T, x1) - r1 == 0)

opti_cons1.set_initial(x1,x0)
opti_cons1.solver('ipopt')

print("\n \n  Résolution avec Casadi")
start_time = time.time()
sol = opti_cons1.solve()
print("Durée : ", time.time()-start_time)
print(sol.value(x1))
'''

'\n# équivalent avec Casadi pour r = 0.2\n\nopti_cons1 = casadi.Opti();\nx1 = opti_cons1.variable(3)\n\nopti_cons1.minimize(var(x1))\nopti_cons1.subject_to(dot(unit.T, x1) - 1 == 0)\nopti_cons1.subject_to(dot(p.T, x1) - r1 == 0)\n\nopti_cons1.set_initial(x1,x0)\nopti_cons1.solver(\'ipopt\')\n\nprint("\n \n  Résolution avec Casadi")\nstart_time = time.time()\nsol = opti_cons1.solve()\nprint("Durée : ", time.time()-start_time)\nprint(sol.value(x1))\n'

In [76]:
'''
# équivalent avec Casadi pour r = 0.15

opti_cons2 = casadi.Opti();
x2 = opti_cons2.variable(3)

opti_cons2.minimize(var(x2))
opti_cons2.subject_to(dot(unit.T, x2) - 1 == 0)
opti_cons2.subject_to(dot(p.T, x2) - r2 == 0)

opti_cons2.set_initial(x2,x0)
opti_cons2.solver('ipopt')

print("\n \n  Résolution avec Casadi")
start_time = time.time()
sol = opti_cons2.solve()
print("Durée : ", time.time()-start_time)
print(sol.value(x2))
'''

'\n# équivalent avec Casadi pour r = 0.15\n\nopti_cons2 = casadi.Opti();\nx2 = opti_cons2.variable(3)\n\nopti_cons2.minimize(var(x2))\nopti_cons2.subject_to(dot(unit.T, x2) - 1 == 0)\nopti_cons2.subject_to(dot(p.T, x2) - r2 == 0)\n\nopti_cons2.set_initial(x2,x0)\nopti_cons2.solver(\'ipopt\')\n\nprint("\n \n  Résolution avec Casadi")\nstart_time = time.time()\nsol = opti_cons2.solve()\nprint("Durée : ", time.time()-start_time)\nprint(sol.value(x2))\n'

On voit que la variance est une fonction croissante du rendement $r$ (on passe de $10^{-2}$ à $10^{-1}$ sur une variation assez faible de rendement, de l'ordre de quelques centièmes).

On retrouve l'idée intuitive que pour un rendement d'investissement élevé (à prix équivalents), les allocations se concentrent davantage sur les actifs qui engendrent du bénéfice, mais donc le risque augmente aussi car on obtient des allocations réparties moins "uniformément" entre les différents actifs.

$(b)$

__Justification du _Two-Fund Theorem_ :__

La frontière efficace, telle que présentée dans l'énoncé, représente les allocations qui engendrent le plus grand retour sur investissement (à un niveau de risque donné). Les investisseurs ont donc intérêt à établir leurs allocations sur cette frontière.

De plus, comme les allocations sont ici des vecteurs, connaissant les allocations de deux acteurs A et B, un investisseur C peut créer son allocation en effectuant une combinaison linéaire des allocations de A et de B.

L'énoncé du théorème vient de la nature de la courbe de frontière efficace : c'est une hyperbole, donc on peut reconstruire un investissement à partir de 2 investissements sur cette hyperbole. Ainsi, un investisseur qui a accès à 2 fonds "optimaux" peut choisir son allocation (en fonction de son aversion au risque) en se ajustant les proportions d'investissement sur ces deux fonds.

$(c)$

*Gemini*

You're asking for the analytical proof of the Two-Fund Theorem, which is the heart of Question 8c. This is a more involved derivation. Here's how we can approach it:

**1. Problem Formulation**

We start with the Markowitz portfolio optimization problem (4) from your document:

```
min  x'Σx
```

Subject to:

```
1'x = 1   (1)
p'x = r   (2)
```

Where:

* `x` is the vector of portfolio weights.
* `Σ` is the covariance matrix.
* `p` is the vector of expected returns.
* `r` is the target expected return.
* `1` is a vector of ones.

**2. Lagrangian**

To solve this constrained optimization problem analytically, we use the method of Lagrange multipliers. We define the Lagrangian function:

```
L(x, λ, μ) = x'Σx + λ(1'x - 1) + μ(p'x - r)
```

Where λ and μ are Lagrange multipliers associated with the constraints (1) and (2), respectively.

**3. First-Order Conditions**

To find the optimal solution, we take the derivative of the Lagrangian with respect to x, λ, and μ and set them equal to zero:

* ∂L/∂x = 2Σx + λ1 + μp = 0   (3)
* ∂L/∂λ = 1'x - 1 = 0   (4)
* ∂L/∂μ = p'x - r = 0   (5)

**4. Solving for x**

From equation (3), we can solve for x:

```
x = -1/2 Σ⁻¹(λ1 + μp)   (6)
```

**5. Substituting into Constraints**

Now, substitute this expression for x into the constraint equations (4) and (5):

* 1'(-1/2 Σ⁻¹(λ1 + μp)) = 1   (7)
* p'(-1/2 Σ⁻¹(λ1 + μp)) = r   (8)

**6. Matrix Notation**

Let's define some helpful notation:

* A = 1'Σ⁻¹1
* B = 1'Σ⁻¹p = p'Σ⁻¹1
* C = p'Σ⁻¹p

Using this, we can rewrite equations (7) and (8) as:

* -1/2 (λA + μB) = 1   (9)
* -1/2 (λB + μC) = r   (10)

**7. Solving for Lagrange Multipliers**

Now we have a system of two linear equations (9) and (10) with two unknowns (λ and μ). We can solve for λ and μ:

```
λ = -2(C - rB) / (AC - B²)
μ = 2(rA - B) / (AC - B²)
```

**8. Substituting Back into x**

Substitute these expressions for λ and μ back into equation (6) to get the optimal portfolio weights x:

```
x = Σ⁻¹ [ (C - rB) / (AC - B²) ] 1 + Σ⁻¹ [ (rA - B) / (AC - B²) ] p
```

**9. Two-Fund Theorem**

Notice that this equation can be rewritten as a linear combination of two portfolios:

```
x = α * Σ⁻¹1 + β * Σ⁻¹p
```

Where α and β are scalar coefficients that depend on A, B, C, and the target return r.

This shows that any optimal portfolio x can be expressed as a combination of two specific portfolios: Σ⁻¹1 and Σ⁻¹p. These two portfolios are the "two funds." By varying α and β, we can generate any portfolio on the efficient frontier.

**In Conclusion**

The analytical solution demonstrates that the optimal portfolio weights are always a linear combination of two specific portfolios, thus proving the Two-Fund Theorem.

# Question 9

In [77]:
r9 = 0.4
s_M = 0.5

# Comme x et s sont liées, on va créer un array 'variables' de taille 6 qui contient la concaténation de x et de s
def var2(variables):
    x = variables[:3]
    return x.T @ sigma @ x

def contrainte_sx(variables):
    x = variables[:3]
    s = variables[3:]
    return -(s + x) #on veut la contrainte inégalité s >= -x

def contrainte_spos(variables):
    s = variables[3:]
    return -s

def contrainte_sM(variables):
    s = variables[3:]
    return unit.T@s - s_M

contraintes9 = ({'type': 'eq', 'fun': lambda variables:  unit.T@variables[:3] -1},
        {'type': 'eq', 'fun': lambda variables: p.T@variables[:3] - r9},
        {'type': 'ineq', 'fun': contrainte_sx},
        {'type': 'ineq', 'fun': contrainte_spos},
        {'type': 'ineq', 'fun': contrainte_sM})

variables0 = np.concatenate((x0, np.zeros(3)), axis=0) #initialisation de x et s
bounds_x = [(None, None)] * 3 #pas d'encadrement de x
bounds_s = [(0, None)]*3 #pour s'assurer que s >= 0
bounds = bounds_s + bounds_x

print('  Méthode SLSQP avec Scipy (r = 0.4, s_M = 0.5)')
start_time = time.time()
res = optimize.minimize(var2, variables0, method='SLSQP', jac=None, bounds=bounds, constraints=contraintes9, options={'disp': True})
print("Allocation :", res.x[:3]) #on n'affiche que x, pas s
print("Durée : ", time.time()-start_time)

  Méthode SLSQP avec Scipy (r = 0.4, s_M = 0.5)
Inequality constraints incompatible    (Exit mode 4)
            Current function value: 0.16486000759578212
            Iterations: 17
            Function evaluations: 230
            Gradient evaluations: 17
Allocation : [0.3        0.19999999 0.50000001]
Durée :  0.019878864288330078


On obtient une variance plus élevée (ce qui est prévisible étant donné que $r$ est plus élevé), mais tout de même moins élevée que celle avec $r = 0,4$ dans la configuration où on ne définit pas la contrainte inégalité : limiter les _shortings_ permet de limiter les fluctuations du marché, et donc d'avoir des variations de prix des actifs moins élevées. Le risque associé diminue donc.

En revanche, les actifs sont davantage distribués : en effet, en limitant les positions courtes, on limite aussi la diversification des allocations et l'écart d'investissement entre les actifs (on ne peut pas compenser les pertes potentielles sur une allocation avec des positions short pour un autre actif).